In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt


# Data PreProcessing

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# Hyperparameters
num_classes = 200           # Tiny ImageNet has 200 classes
batch_size = 32
num_epochs = 10
learning_rate = 0.001

# Path to your Tiny ImageNet data (should have 'train' and 'val' folders)
data_dir = r'\Users\aabla\OneDrive\Desktop\COGS181\COGS181-Project\tiny-imagenet-200'

# Define data transforms for training and validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomResizedCrop(224),  # VGG expects 224x224 input
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],  # Standard normalization values for pre-trained models
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

# Load the datasets assuming the folder structure is:
#   tiny-imagenet-200/train/<class_name>/...
#   tiny-imagenet-200/val/<class_name>/...
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val', 'test']}

# Create dataloaders
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                              batch_size=batch_size,
                                              shuffle=True,
                                              num_workers=4)
               for x in ['train', 'val', 'test']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes

cpu


# VGG16 Model Implementation

In [4]:
# Load a pre-trained VGG16 model
model = models.vgg16(pretrained=True)

# Modify the final fully connected layer to match the number of classes in Tiny ImageNet
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
model = model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


c:\Users\aabla\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\aabla\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\aabla/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [00:13<00:00, 41.8MB/s] 


# Training Loop

In [5]:
for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluation mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            # Forward pass; track history only if in training phase
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # Backward pass + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]
        print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    print('-' * 30)

Epoch 1/10


KeyboardInterrupt: 

# Feature Visualization of Each Layer

In [ ]:
# Dictionary to store activations
activations = {}

# Hook function to capture the output of a layer
def get_activation(name):
    def hook(model, input, output):
        activations[name] = output.detach()
    return hook

# Register hooks on each layer in the VGG16 feature extractor (the convolutional layers)
for idx, layer in enumerate(model.features):
    layer.register_forward_hook(get_activation(f'features_{idx}'))

# Get a sample batch from the validation set (or use a single image)
model.eval()
sample_inputs, _ = next(iter(dataloaders['val']))
sample_inputs = sample_inputs.to(device)

# Forward pass to capture activations
_ = model(sample_inputs)

# Visualize the activations for each registered layer for the first image in the batch
for layer_name, activation in activations.items():
    # activation shape: [batch_size, channels, height, width]
    act = activation[0].cpu()  # Take the first image in the batch
    num_channels = act.shape[0]
    # Visualize up to 8 feature maps from the layer to keep the plots readable
    n_cols = min(num_channels, 8)
    plt.figure(figsize=(n_cols * 2, 2))
    for i in range(n_cols):
        plt.subplot(1, n_cols, i + 1)
        plt.imshow(act[i].numpy(), cmap='viridis')
        plt.title(f'{layer_name}\nChannel {i}')
        plt.axis('off')
    plt.suptitle(f'Activations from {layer_name}')
    plt.show()